# Preliminary Testing

In [1]:
import wikipediaapi
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import tarfile
import re
import urllib.parse
from time import sleep
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia('MyMovieScript/1.0 (danja.z@bluewin.ch) requests/2.26.0 BeautifulSoup/4.10.0')

# Define a user-agent for requests
headers = {
    'User-Agent': 'MyMovieScript/1.0 (danja.z@bluewin.ch) requests/2.26.0 BeautifulSoup/4.10.0'
}

In [3]:
# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_non-fiction_works_made_into_feature_films"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table', {'class': 'wikitable'})

# Create a list to store DataFrames
dfs = []

# Loop through each table and read it into a DataFrame
for i, table in enumerate(tables):
    df = pd.read_html(str(table))[0]
    # Optionally add a column to identify the table source
    df['Source Table'] = f'Table {i+1}'  # Label each table
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a CSV file
combined_df.to_csv('all_non_fiction_films.csv', index=False)

print("Data from all tables has been successfully extracted and saved to 'all_non_fiction_films.csv'.")


Data from all tables has been successfully extracted and saved to 'all_non_fiction_films.csv'.


/var/folders/_t/wz53gqgx2jl97snjc7ndwdd80000gn/T/ipykernel_45356/18779703.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/_t/wz53gqgx2jl97snjc7ndwdd80000gn/T/ipykernel_45356/18779703.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/_t/wz53gqgx2jl97snjc7ndwdd80000gn/T/ipykernel_45356/18779703.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/_t/wz53gqgx2jl97snjc7ndwdd80000gn/T/ipykernel_45356/18779703.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated

In [4]:
file_path = 'all_non_fiction_films.csv'
df = pd.read_csv(file_path)

# Step 1: Combine Non-fiction work(s), Book, Essay/article into a single column called book_name
df['book_name'] = df['Non-fiction work(s)'].fillna('') + df['Book'].fillna('') + df['Essay/article'].fillna('')
df['book_name'] = df['book_name'].replace('', pd.NA)

# Step 2: Drop the Subject column
df.drop(columns=['Subject'], inplace=True)

# Initialize new columns
df['book_year'] = pd.NA
df['author'] = pd.NA
df['movie_year'] = pd.NA

# Step 3: Process entries where Source Table == 1
def split_source_table_1(row):
    if row['Source Table'] == 1:
        # Match the book name, year, and author
        print('yes')
        match = re.match(r'^(.*) \((\d{4})\), (.*)$', row['book_name'].strip())
        if match:
            row['book_name'] = match.group(1).strip()  # Book name
            row['author'] = match.group(3).strip()      # Author name
            row['book_year'] = match.group(2).strip()  # Book year
            
    return row

df = df.apply(split_source_table_1, axis=1)

# Step 4: Extract movie_year from Film adaptation(s) where applicable
def extract_movie_year(row):
    if pd.notna(row['book_name']):
        match = re.search(r'\((\d{4})\)', row['Film adaptation(s)'])
        if match:
            row['movie_year'] = match.group(1).strip()
    return row

df = df.apply(extract_movie_year, axis=1)

# Step 5: Rearranging the columns for better readability (optional)
columns_order = ['book_name', 'book_year', 'author', 'movie_year', 'Film adaptation(s)', 'Source Table']
df = df[columns_order]

# Save the cleaned DataFrame to a new CSV file
df.to_csv('cleaned_file_non_fiction.csv', index=False)

print("Data cleaning completed and saved to", )


Data cleaning completed and saved to


In [51]:
# Define column names
column_names = [
    'movie_id',
    'movie_cluster',
    'movie_name',
    'release_date',
    'duration',
    'budget',
    'language',
    'country',
    'genres'
]

# Load the TSV file into a DataFrame with the specified column names
df_csu = pd.read_csv('./MovieSummaries/movie.metadata.tsv', delimiter='\t', header=None, names=column_names)

# Convert 'duration' and 'budget' to numeric
df_csu['duration'] = pd.to_numeric(df_csu['duration'], errors='coerce')
df_csu['budget'] = pd.to_numeric(df_csu['budget'], errors='coerce')



display(df_csu)


,movie_id,movie_cluster,movie_name,release_date,duration,budget,language,country,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [71]:
df_non_fiction = pd.read_csv('./Wikipedia_files/cleaned_file_non_fiction.csv')
count_common_indexing = df_csu['movie_name'].isin(df_non_fiction['Film adaptation(s)']).sum()
print(f"Count of common elements using indexing non fiction books: {count_common_indexing}")

df_children = pd.read_csv('./Wikipedia_files/children_cleaned.csv')
count_common_indexing = df_csu['movie_name'].isin(df_children['Film adaptation(s)']).sum()
print(f"Count of common elements using indexing, childrens books: {count_common_indexing}")

df_arts = pd.read_csv('./Wikipedia_files/arts.csv')
count_common_indexing = df_csu['movie_name'].isin(df_arts['Film']).sum()
print(f"Count of common elements using indexing, arts: {count_common_indexing}")

df_crime = pd.read_csv('./Wikipedia_files/crime.csv')
count_common_indexing = df_csu['movie_name'].isin(df_crime['Film']).sum()
print(f"Count of common elements using indexing, crimes: {count_common_indexing}")


Count of common elements using indexing non fiction books: 272
Count of common elements using indexing, childrens books: 705
Count of common elements using indexing, arts: 71
Count of common elements using indexing, crimes: 114


In [7]:
# Connect to the SQLite database
conn = sqlite3.connect('books_films_reviews.sqlite3')  # Replace with your file name

# Query to list all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query
tables = conn.execute(tables_query).fetchall()


# Display the list of tables
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:


In [8]:
# Export each table to a CSV file
for table in tables:
    table_name = table[0]
    
    # Query to select all data from the table
    query = f"SELECT * FROM {table_name};"
    
    # Load data into a DataFrame
    df = pd.read_sql_query(query, conn)
    
    # Define the CSV filename
    csv_file_name = f"{table_name}.csv"
    
    # Export the DataFrame to a CSV file
    df.to_csv(csv_file_name, index=False)
    print(f"Exported {table_name} to {csv_file_name}")

# Close the connection
conn.close()

In [10]:
df_wiki = pd.read_csv('wiki_book_movie_ids_matching.csv')
count_common_indexing = df_csu['movie_name'].isin(df_wiki['FilmTitle']).sum()
print(f"Count of common elements using indexing: {count_common_indexing}")

Count of common elements using indexing: 2328


In [70]:
print(df_wiki.columns)

Index(['index', 'id', 'BookTitle', 'BookYear', 'Author', 'FilmTitle',
       'FilmYear', 'full name', 'tmdb_id', 'id_goodreads', 'imdbid'],
      dtype='object')


In [73]:
#Let's make one big dataframe
df_non_fiction.rename(columns={'Film adaptation(s)': 'movie_name'}, inplace=True)
df_children.rename(columns={'Film adaptation(s)': 'movie_name'}, inplace=True)
df_arts.rename(columns={'Film': 'movie_name', 'Date': 'movie_year'}, inplace=True)
df_crime.rename(columns={'Film': 'movie_name', 'Date': 'movie_year'}, inplace=True)
df_wiki.rename(columns={'FilmTitle': 'movie_name', 'FilmYear': 'movie_year'}, inplace=True)
df_csu['movie_year'] = df_csu['release_date'].str[:4]

print('csu',df_csu.columns)
print('non fiction', df_non_fiction.columns)
print('childre', df_children.columns)
print('arts', df_arts.columns)
print('crime', df_crime.columns)
print('wiki', df_wiki.columns)



csu Index(['movie_id', 'movie_cluster', 'movie_name', 'release_date', 'duration',
       'budget', 'language', 'country', 'genres', 'movie_year'],
      dtype='object')
non fiction Index(['book_name', 'book_year', 'author', 'movie_year', 'movie_name',
       'Source Table'],
      dtype='object')
childre Index(['book_name', 'book_year', 'author', 'movie_year', 'movie_name',
       'Source Table'],
      dtype='object')
arts Index(['movie_name', 'movie_year', 'Director', 'Country', 'Source work',
       'Author', 'Date.1', 'Type', 'Basis', 'Source Table'],
      dtype='object')
crime Index(['#', 'movie_name', 'movie_year', 'Director', 'Country', 'Source work',
       'Author', 'Date.1', 'Type', 'Source Table', 'TV', 'Date.2', 'Country.1',
       'Basis'],
      dtype='object')
wiki Index(['index', 'id', 'BookTitle', 'BookYear', 'Author', 'movie_name',
       'movie_year', 'full name', 'tmdb_id', 'id_goodreads', 'imdbid'],
      dtype='object')


In [57]:
display(df_csu)

,movie_id,movie_cluster,movie_name,release_date,duration,budget,language,country,genres,movie_year
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983
...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",2011
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",2011
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",1972
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",1992


In [74]:
#lets make a new dataset that contains all matches
for df in [df_csu, df_non_fiction, df_children, df_arts, df_crime]:
    df['movie_year'] = pd.to_numeric(df['movie_year'], errors='coerce').fillna(0).astype(int)
# Merge all five DataFrames on 'movie_name' and 'movie_year', using an inner join to avoid duplicates.
merged_df_non_fiction = df_csu.merge(df_non_fiction, on=['movie_name', 'movie_year'], how='inner')
merged_children = df_csu.merge(df_children, on=['movie_name', 'movie_year'], how='inner') 
merged_arts = df_csu.merge(df_arts, on=['movie_name', 'movie_year'], how='inner') 
merged_crime = df_csu.merge(df_crime, on=['movie_name', 'movie_year'], how='inner')
merged_wiki = df_csu.merge(df_wiki, on=['movie_name', 'movie_year'], how='inner')


In [79]:
display(merged_wiki)

,movie_id,movie_cluster,movie_name,release_date,duration,budget,language,country,genres,movie_year,index,id,BookTitle,BookYear,Author,full name,tmdb_id,id_goodreads,imdbid
0,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""...",1964,1279,1855,Mary Poppins,1934,P. L. Travers,Mary Poppins 1964,433,152380,tt0058331
1,1369204,/m/04x8zs,Juarez,1939,NaN,125.0,"{""/m/02h40lc"": ""English Language"", ""/m/06nm1"":...","{""/m/09c7w0"": ""United States of America""}","{""/m/04xvh5"": ""Costume drama"", ""/m/03bxz7"": ""B...",1939,1504,2217,The Phantom Crown: The Story of Maximilian & C...,1934,0,Juarez 1939,78318,10546105,tt0031516
2,164388,/m/015wmt,The Great Santini,1979-10-26,4702575.0,115.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02h8pkk"": ""Family Drama"", ""/m/07s9rl0"": ""...",1979,883,1253,The Great Santini,1976,Pat Conroy,The Great Santini 1979,21887,1110004,tt0079239
3,19095614,/m/04yc5nd,The Castle,1968,NaN,88.0,"{""/m/04306rv"": ""German Language""}","{""/m/082fr"": ""West Germany""}","{""/m/02n4kr"": ""Mystery"", ""/m/07s9rl0"": ""Drama""}",1968,395,564,The Castle,1926,Franz Kafka,The Castle 1968,197962,152864,tt0063563
4,13613357,/m/03cbvzw,Grand Canary,1934-07-20,NaN,78.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02l7c8"": ""Romance Film"", ""/m/07s9rl0"": ""D...",1934,867,1229,Grand Canary,1933,A. J. Cronin,Grand Canary 1934,432684,37941573,tt0025196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,5440243,/m/0dl_jw,Beau Geste,1939,NaN,112.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/03k9fj"": ""Adventure"", ""/m/01j6zc"": ""Forei...",1939,219,300,Beau Geste,1924,P. C. Wren,Beau Geste 1939,36851,997335,tt0031088
1976,33834117,/m/0hnffsw,Ivanhoe,1913-07-01,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/06ppq"": ""Silent film"", ""/m/07s9rl0"": ""Dra...",1913,1058,1490,Ivanhoe,1820,Sir Walter Scott,Ivanhoe 1913,404848,6440,tt0003022
1977,1840870,/m/0603hj,Rising Sun,1993-07-30,107198790.0,129.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/0lsxr"": ""Crime F...",1993,1653,2438,Rising Sun,1992,Michael Crichton,Rising Sun 1993,7007,762145,tt0107969
1978,1832735,/m/05_k34,The Deluge,1974,NaN,315.0,"{""/m/05qqm"": ""Polish Language""}","{""/m/05qhw"": ""Poland""}","{""/m/082gq"": ""War film""}",1974,557,808,The Deluge,1886,Henryk Sienkiewicz,The Deluge 1974,36627,25239218,tt0072021


In [84]:
merged_wiki.to_csv('books_movies_matches.csv')